In [ ]:
import xarray as xr

from sklearn.externals import joblib
import holoviews as hv
hv.extension('bokeh')

# Compute PCA

In [ ]:
data3d = "../data/ngaqua/3d/*.nc"
data2d = "../data/ngaqua/2d/*.nc"
weight = "../data/processed/ngaqua/w.nc"


def load_data(data3d = "../data/ngaqua/3d/*.nc",
    data2d = "../data/ngaqua/2d/*.nc",
    weight = "../data/processed/ngaqua/w.nc",
    **kwargs):
    """Load and merge 2d and 3d datasets
    """
    D = xr.open_mfdataset(data3d)
    D2 = xr.open_mfdataset(data2d)
    w = xr.open_dataarray(weight)

    D = D.merge(D2, join='inner')

    return D, w


In [ ]:

pca_output = joblib.load("../data/ml/ngaqua/pca.pkl")

mod  = pca_output['model']
D, w = load_data(**pca_output)

sig = D.std(['x', 'y', 'time']).load()
mu = D.mean(['x', 'y', 'time']).load()

In [ ]:
pca = mod.named_steps['pca']

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from xnoah.sklearn import Stacker, Weighter, Normalizer, Select

In [ ]:
pipe.fit(D)

In [ ]:
comp = pipe.named_steps['pca'].components_

In [ ]:
%%opts Curve[invert_axes=True]
hv.HoloMap({k: hv.Curve(y/np.sqrt(w)) for k,y in enumerate(comp)})

The PCA modes are pretty noisy. Maybe the MCA modes are better.

# MCA

In [ ]:
mca = joblib.load("../data/ml/ngaqua/mca.pkl")

In [ ]:
mca_mod = mca.named_steps['mca']

In [ ]:
mca_mod.x_components_.shape

In [ ]:
hv.HoloMap({k: hv.Curve(c) for k, c in enumerate(mca_mod.x_components_.T)})

# Linear regession

In [ ]:
lm_data = joblib.load("../data/ml/ngaqua/linear_model.pkl")

In [ ]:
lm_data['test_score']